In [ ]:
using Distributed
addprocs(12)

@everywhere include("problem.jl")
@everywhere include("../fileIO.jl")

@everywhere using LinearAlgebra, Random, Convex, Mosek, MosekTools

# Manipulation System
Mixed-integer implementation of grasp optimization from _Task-Oriented Quality Measures for Dextrous Grasping_, Haschke _et al._ (2005)

In [ ]:
@everywhere function make_dataset(jj;Ndata=500)
    Random.seed!(jj)

    # params
    mu_min, mu_max = 0.1, 0.1
    h_min, h_max = 2, 2
    r_min, r_max = 1, 1
    w_std = 10

    N_v = 5 #number of points to choose from
    N_h = 6
    num_grasps = 4

    params = Dict("N_v"=>N_v, "N_h"=>N_h,
                    "num_grasps"=>num_grasps,
                    "mu_min"=>mu_min, "mu_max"=>mu_max,
                    "h_min"=>h_min, "h_max"=>h_max,
                    "r_min"=>r_min, "r_max"=>r_max,
                    "w_std"=>w_std)

    UB = upperBoundManipulationProb()[1]
    probdata = Dict("params"=>params, "UB"=>UB)

    # Problem
    as = []
    fs = []
    Ys = []
    hs = []
    rs = []
    ws = []
    mus = []
    J = []
    solution_times = []
    node_counts = []
    ii = 0

    while ii < Ndata
        h = h_min + rand()*(h_max-h_min)
        r = r_min + rand()*(r_max-r_min)
        w = w_std*randn(12)
        w = exp.(w)/sum(exp.(w))
        # @show sum(w)
        mu = mu_min + rand()*(mu_max-mu_min)
        
        # Solve Problem        
        a,f,y,cost,solve_time,node_count,solver_success = solveManipulationProb(
            N_v,N_h,num_grasps,w,h,r,mu)

        # Save all the data
        if solver_success
            push!(as, a)
            push!(fs, f)
            push!(Ys, y)
            push!(hs, h)
            push!(rs, r)
            push!(ws, w)
            push!(mus, mu)
            push!(J, cost)
            push!(solution_times, solve_time)
            push!(node_counts,node_count)
            ii += 1
        end
    end
    
    base = "data/testdata"
    filename = string(base, string(jj), ".jld")

    training_data = Dict("a"=>as, "f"=>fs, "Y"=>Ys,
        "J"=>J,
        "h"=>hs, "r"=>rs, "ws"=>ws, "mu"=>mus,
        "solve_time"=>solution_times,
        "node_count"=>node_counts,
        "prob"=>probdata)
    writeTrainingData(filename, training_data)
    return
end

In [ ]:
Nsets = 20
Ndata = 500
dir_idxs = [ii for ii in 1:Nsets]
time_parallel = @elapsed results = @showprogress pmap(make_dataset, dir_idxs)    